<a href="https://colab.research.google.com/github/dinhkhuong/Finetune-MLs/blob/main/Fine_tune_gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [ ]:
import os

import nltk
from nltk.tokenize import sent_tokenize
from datasets import load_dataset, Split
from transformers import ( Trainer, AutoModelForCausalLM, AutoTokenizer,TrainingArguments, pipeline )

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls "/content/drive/My Drive/Colab Notebooks/Finetune MLs/texts"

alice28-1476.sent.csv  alice28-1476.txt


In [ ]:
localfolder = '/content/drive/My Drive/Colab Notebooks/Finetune MLs/texts'


In [ ]:
with open('/content/drive/My Drive/Colab Notebooks/Finetune MLs/texts/alice28-1476.txt', 'r') as f:
    alice = ''.join(f.readlines()[:3704])

The intro of this text is manually removed. For futher optional automation texts removal of the extra lines by setting the real start and end
lines of each text in a configuration file (lines.cfg)
Ex:


```
#
1 text_cfg = """fname,start,end
2 alice28-1476.txt,104,3704"""
3 bytes_written = open(os.path.join(localfolder, 'lines.cfg'), 'w').write(text_cfg)
```



In [ ]:
print(alice[:200])

                ALICE'S ADVENTURES IN WONDERLAND

                          Lewis Carroll

               THE MILLENNIUM FULCRUM EDITION 2.8




                            CHAPTER I

                


In [ ]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
corpus_alice = sent_tokenize(alice)

In [ ]:
len(corpus_alice)

1612

In [ ]:
corpus_alice[2]

'There was nothing so VERY remarkable in that; nor did Alice\nthink it so VERY much out of the way to hear the Rabbit say to\nitself, `Oh dear!'

There's still original line breaks and quotation mark in the corpus, need to remove or pick an propriate quote mark and comma to avoid misinterpreted as
separation chars when put them in CSV file, and (/) is chosen

In [ ]:
os.listdir(localfolder)

['alice28-1476.txt', 'alice28-1476.sent.csv']

In [ ]:
def sentence_tokenize(source, quote_char='\\', sep_char=',',
                      include_header=True, include_source=True,
                      extensions=('txt'), **kwargs):
    nltk.download('punkt')
    # If source is a folder, goes through all files inside it
    # that match the desired extensions ('txt' by default)
    if os.path.isdir(source):
        filenames = [f for f in os.listdir(source)
                     if os.path.isfile(os.path.join(source, f)) and
                        os.path.splitext(f)[1][1:] in extensions]
    elif isinstance(source, str):
        filenames = [source]

    # If there is a configuration file, builds a dictionary with
    # the corresponding start and end lines of each text file
    config_file = os.path.join(source, 'lines.cfg')
    config = {}
    if os.path.exists(config_file):
        with open(config_file, 'r') as f:
            rows = f.readlines()

        for r in rows[1:]:
            fname, start, end = r.strip().split(',')
            config.update({fname: (int(start), int(end))})

    new_fnames = []
    # For each file of text
    for fname in filenames:
        # If there's a start and end line for that file, use it
        try:
            start, end = config[fname]
        except KeyError:
            start = None
            end = None

        # Opens the file, slices the configures lines (if any)
        # cleans line breaks and uses the sentence tokenizer
        with open(os.path.join(source, fname), 'r') as f:
            contents = (''.join(f.readlines()[slice(start, end, None)])
                        .replace('\n', ' ').replace('\r', ''))
        corpus = sent_tokenize(contents, **kwargs)

        # Builds a CSV file containing tokenized sentences
        base = os.path.splitext(fname)[0]
        new_fname = f'{base}.sent.csv'
        new_fname = os.path.join(source, new_fname)
        with open(new_fname, 'w') as f:
            # Header of the file
            if include_header:
                if include_source:
                    f.write('sentence,source\n')
                else:
                    f.write('sentence\n')
            # Writes one line for each sentence
            for sentence in corpus:
                if include_source:
                    f.write(f'{quote_char}{sentence}{quote_char}{sep_char}{fname}\n')
                else:
                    f.write(f'{quote_char}{sentence}{quote_char}\n')
        new_fnames.append(new_fname)

    # Returns list of the newly generated CSV files
    return sorted(new_fnames)

In [ ]:
new_fnames = sentence_tokenize(localfolder)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
new_fnames

['/content/drive/My Drive/Colab Notebooks/texts/alice28-1476.sent.csv']

In [ ]:
dataset = load_dataset(path='csv', data_files=['/content/drive/My Drive/Colab Notebooks/Finetune MLs/texts/alice28-1476.sent.csv'], quotechar='\\', split=Split.TRAIN)

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset[6]

{'sentence': 'In another moment down went Alice after it, never once considering how in the world she was to get out again.',
 'source': 'alice28-1476.txt'}

In [ ]:
shuffled_dataset = dataset.shuffle(seed=42)
split_dataset = shuffled_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset, test_dataset = split_dataset['train'], split_dataset['test']

In [ ]:
auto_tokenizer = AutoTokenizer.from_pretrained('gpt2')
def tokenize(row):
    return auto_tokenizer(row['sentence'])

GPT-2 uses a Byte-Pair Encoding (BPE) instead of WordPiece.

No need padding for generative tasks.

In [ ]:
tokenized_train_dataset = train_dataset.map(tokenize, remove_columns=['source', 'sentence'], batched=True)
tokenized_test_dataset = test_dataset.map(tokenize, remove_columns=['source', 'sentence'], batched=True)

Map:   0%|          | 0/1289 [00:00<?, ? examples/s]

Map:   0%|          | 0/323 [00:00<?, ? examples/s]

In [ ]:
list(map(len, tokenized_train_dataset[0:6]['input_ids']))

[9, 28, 20, 9, 34, 29]

Without padding, the sentences have varied lengths. The solution is to "pack sequence" by simply concatenating the inputs together and then chunking them into blocks

In [ ]:
#from https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_clm.py
def group_texts(examples, block_size=128):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_train_dataset = tokenized_train_dataset.map(group_texts, batched=True)
lm_test_dataset = tokenized_test_dataset.map(group_texts, batched=True)
lm_train_dataset.set_format(type='torch')
lm_test_dataset.set_format(type='torch')

Map:   0%|          | 0/1289 [00:00<?, ? examples/s]

Map:   0%|          | 0/323 [00:00<?, ? examples/s]

In [ ]:
print(lm_train_dataset[0]['input_ids'])

tensor([   63,  2437,   466,   345,   760,   314,  1101,  8805,  8348,   464,
         2677,  3114,  7296,  6819,   379,   262,  2635, 25498,    11,   508,
          531,   287,   257,  1877,  3809,    11,  4600,  7120, 25788,  1276,
         3272,    12,  1069,  9862, 12680,  4973,  2637,  1537,   611,   314,
         1101,   407,   262,   976,    11,   262,  1306,  1808,   318,    11,
         5338,   287,   262,   995,   716,   314,    30,   464,   360,   579,
         1076,  6364,  4721,   465,  2951,    13,    63,  1026,   373,   881,
        21289,   272,   353,   379,  1363,  4032,  1807,  3595, 14862,    11,
         4600, 12518,   530,  2492,   470,  1464,  3957,  4025,   290,  4833,
           11,   290,   852,  6149,   546,   416, 10693,   290, 33043,    13,
         1870, 14862,   373,   523,   881, 24776,   326,   673,  4966,   572,
          379,  1752,   287,   262,  4571,   340,  6235,   284,    11,  1231,
         2111,   284,  4727,   262,  7457,   340,   550,   925])

In [ ]:
len(lm_train_dataset), len(lm_test_dataset)

(239, 56)

In [ ]:
model = AutoModelForCausalLM.from_pretrained('gpt2')
print(model.__class__)

<class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'>


GPT-2's tokenizer does not include a special padding token by default but you may add it if needed. If you do add any tokens to the vocabulary, though, you also need to let the model know it using resize_token_embeddings

```
# model.resize_token_embeddings(len(auto_tokenizer))
```



In [ ]:
training_args = TrainingArguments(
    output_dir='output',
    num_train_epochs=1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=8,
    evaluation_strategy='steps',
    eval_steps=50,
    logging_steps=50,
    gradient_accumulation_steps=4,
    prediction_loss_only=True,
)

trainer = Trainer(model=model,
                  args=training_args,
                  train_dataset=lm_train_dataset,
                  eval_dataset=lm_test_dataset)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
50,3.559500,3.337878


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 50, 'do_sample': True}


TrainOutput(global_step=59, training_loss=3.5460024041644598, metrics={'train_runtime': 754.4326, 'train_samples_per_second': 0.317, 'train_steps_per_second': 0.078, 'total_flos': 15416229888000.0, 'train_loss': 3.5460024041644598, 'epoch': 0.9874476987447699})

In [ ]:
trainer.evaluate()

{'eval_loss': 3.33121657371521,
 'eval_runtime': 50.3421,
 'eval_samples_per_second': 1.112,
 'eval_steps_per_second': 0.139,
 'epoch': 0.9874476987447699}

In [ ]:
device_index = model.device.index if model.device.type != 'cpu' else -1
gpt2_gen = pipeline('text-generation', model=model, tokenizer=auto_tokenizer, device=device_index)

In [ ]:
base_text = """
Alice got bigger.
"""

In [ ]:
result = gpt2_gen(base_text, max_length=150)
print(result[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Alice got bigger.
Alice had not seen it before; nor had she seen any thing yet, except a piece of wood, a piece of stone, the shape of the body of which was as like a monkey, and was the same size as the two dolls.
Alice felt very curious about that, and she wanted to leave the room as soon as possible, after she had taken out her pencil as it was held.
(I think.) "Alice, would you be pleased to see me at that, if you liked, as soon as you got out of the garden?"
(She did as she was told, and I should like to talk by you.) "I suppose I will get to the garden, and to your
